In [1]:
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(color_codes=True)

from IPython.display import display

import re
patern1= "\(([ а-яa-zА-ЯA-Z0-9 ]*)\)"
patern2= "\([ а-яa-zА-ЯA-Z0-9 ]*\)"

file="КЦП_магистратура_Академия.xlsx"
#xl = pd.ExcelFile(file)
df_mos_origin = pd.read_excel(file, sheet_name = "магистратура")[6:452]#,header=3)
#display(df_mos_origin.head(10))
df_mos = df_mos_origin[['Unnamed: 0','Unnamed: 1']]
df_mos = df_mos.reset_index(drop=True)
df_mos.columns = ['код','программа']
#display(df_mos.head(3))
#display(df_mos.tail(10))
df_mos = df_mos[~df_mos['программа'].str.contains('по программе|по совокупности')]
df_code = df_mos.dropna()
code_index = df_code.index
code = df_code.drop_duplicates().reset_index(drop=True)
code.columns = ['код','стандарт']
df_mos = df_mos.fillna(method='ffill').drop(code_index)
df_mos['программа'] = df_mos['программа'].map(lambda x: re.sub('[ё]', 'е', x))
df_mos['подразделение'] = df_mos['программа'].map(lambda x: re.findall(patern1, x )[-1])
df_mos['программа'] = df_mos['программа'].map(lambda x: x.replace( re.findall(patern2, x )[-1],'').lstrip().rstrip() )
df_mos['программа'] = df_mos['программа'].map(lambda x: re.sub('\d+.', "", x).lstrip().rstrip() )
#df_mos = df_mos.reset_index(drop=True)
df_mos = df_mos.drop_duplicates().reset_index(drop=True)
df_mos = pd.merge(df_mos, code, on='код')
df_mos = df_mos[['код','стандарт','программа','подразделение']]
display(df_mos.tail(5))



,код,стандарт,программа,подразделение
149,42.04.01,Реклама и связи с общественностью,Цифровые коммуникации и новые медиа,ИОН
150,42.04.02,Журналистика,Журналистика и медиатехнологии,ИГСУ
151,46.04.01,История,Социально-экономическая история России и мира,ИОН
152,46.04.01,История,Россия и Франция: историко-культурные связи,ИОН
153,46.04.01,История,Политическая и культурная история Европы,ИОН


In [2]:
#### 
####  Условие отбора программ
####
#df = df_mos
df = df_mos[(df_mos['код'] == "38.04.08")|
            (df_mos['код'] == "38.04.01")
           ].copy().reset_index(drop = True)
unique_name = df['программа'].unique()
unique_sort = np.sort(unique_name)
display(unique_sort)
for i in unique_sort:
    display(df_mos[df_mos['программа'] == i].reset_index(drop = True))

array(['Банки, финансы, инвестиции',
       'Государственное регулирование экономики',
       'Денежно-кредитное и финансовое регулирование экономики',
       'Корпоративная экономика', 'Поведенческая экономика',
       'Прикладные корпоративные финансы',
       'Регулирование финансовых рынков и институтов',
       'Системы больших данных в экономике',
       'Управление инвестициями и инновациями в экономике',
       'Финансовая дипломатия',
       'Финансовый контроль и государственный аудит',
       'Финансовый контроль и государственный аудит (российско-французская программа)',
       'Финансы и инвестиции', 'Финансы и стратегии устойчивого развития',
       'Финансы: международные технологии учета и управления',
       'Финансы: учет, анализ и аудит', 'Цифровая экономика',
       'Экономика и контрольно-надзорная деятельность',
       'Экономика и право', 'Экономика и финансы',
       'Экономика медиаиндустрии'], dtype=object)

,код,стандарт,программа,подразделение
0,38.04.01,Экономика,"Банки, финансы, инвестиции",ФФБ


,код,стандарт,программа,подразделение
0,38.04.01,Экономика,Государственное регулирование экономики,ИГСУ


,код,стандарт,программа,подразделение
0,38.04.08,Финансы и кредит,Денежно-кредитное и финансовое регулирование э...,ФФБ


,код,стандарт,программа,подразделение
0,38.04.01,Экономика,Корпоративная экономика,ЭФ ЭМИТ


...

,код,стандарт,программа,подразделение
0,38.04.01,Экономика,Цифровая экономика,ЭФ ЭМИТ


,код,стандарт,программа,подразделение
0,38.04.01,Экономика,Экономика и контрольно-надзорная деятельность,ЭФ ЭМИТ


,код,стандарт,программа,подразделение
0,38.04.01,Экономика,Экономика и право,ЭФ ЭМИТ


,код,стандарт,программа,подразделение
0,38.04.01,Экономика,Экономика и финансы,ЭФ ЭМИТ


,код,стандарт,программа,подразделение
0,38.04.01,Экономика,Экономика медиаиндустрии,ЭФ ЭМИТ


In [2]:
#### 
####  Условие отбора программ
####
#df = df_mos
df = df_mos[(df_mos['код'] == "38.04.08")|
            (df_mos['код'] == "38.04.01")
           ].copy().reset_index(drop = True)
unique_name = df['программа'].unique()
unique_sort = np.sort(unique_name)
display(unique_sort)
for i in unique_sort:
    display(df_mos[df_mos['программа'] == i].reset_index(drop = True))

array(['Банки, финансы, инвестиции',
       'Государственное регулирование экономики',
       'Денежно-кредитное и финансовое регулирование экономики',
       'Корпоративная экономика', 'Поведенческая экономика',
       'Прикладные корпоративные финансы',
       'Регулирование финансовых рынков и институтов',
       'Системы больших данных в экономике',
       'Управление инвестициями и инновациями в экономике',
       'Финансовая дипломатия',
       'Финансовый контроль и государственный аудит',
       'Финансовый контроль и государственный аудит (российско-французская программа)',
       'Финансы и инвестиции', 'Финансы и стратегии устойчивого развития',
       'Финансы: международные технологии учета и управления',
       'Финансы: учет, анализ и аудит', 'Цифровая экономика',
       'Экономика и контрольно-надзорная деятельность',
       'Экономика и право', 'Экономика и финансы',
       'Экономика медиаиндустрии'], dtype=object)

,код,стандарт,программа,подразделение
0,38.04.01,Экономика,"Банки, финансы, инвестиции",ФФБ


,код,стандарт,программа,подразделение
0,38.04.01,Экономика,Государственное регулирование экономики,ИГСУ


,код,стандарт,программа,подразделение
0,38.04.08,Финансы и кредит,Денежно-кредитное и финансовое регулирование э...,ФФБ


,код,стандарт,программа,подразделение
0,38.04.01,Экономика,Корпоративная экономика,ЭФ ЭМИТ


...

,код,стандарт,программа,подразделение
0,38.04.01,Экономика,Цифровая экономика,ЭФ ЭМИТ


,код,стандарт,программа,подразделение
0,38.04.01,Экономика,Экономика и контрольно-надзорная деятельность,ЭФ ЭМИТ


,код,стандарт,программа,подразделение
0,38.04.01,Экономика,Экономика и право,ЭФ ЭМИТ


,код,стандарт,программа,подразделение
0,38.04.01,Экономика,Экономика и финансы,ЭФ ЭМИТ


,код,стандарт,программа,подразделение
0,38.04.01,Экономика,Экономика медиаиндустрии,ЭФ ЭМИТ
